In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
# WIDGETS
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display

In [3]:
# plot
import matplotlib.pyplot as plt
import bg_mpl_stylesheet
from bg_mpl_stylesheet.bg_mpl_stylesheet import bg_mpl_style
plt.style.use(bg_mpl_style)
plt.ion()


In [4]:
def get_x_fitting_range(x, y, xrange: list):
    # x bound
    if xrange[0] == "min":
        xmin = float(x.min())
    else:
        xmin = float(xrange[0])
    if xrange[1] == "max":
        xmax = float(x.max())
    else:
        xmax = float(xrange[1])
    assert all([isinstance(xmin, float), isinstance(xmax, float)])
    
    bool_range = np.logical_and((x <= xmax), (x >= xmin))
    _x = x[bool_range]
    _y = list()
    for i, yi in enumerate(y):
        yi = yi[bool_range]
        _y.append(yi)
    return _x, np.array(_y)

In [5]:
def xy_dim_labels(xy_lable: str):
    dim_y = xy_lable[0]
    dim_x = xy_lable[1]
    if dim_y == 'i':
        dim_y = 'I (a.u.)'
    elif dim_y == 's':
        dim_y = 'S ($\mathrm{\AA^{-1}})$'
    elif dim_y == 'f':
        dim_y = 'F ($\mathrm{\AA^{-1}})$'
    elif dim_y == 'g':
        dim_y = 'G ($\mathrm{\AA^{-2}})$'
    if dim_x == 'q':
        dim_x = 'Q ($\mathrm{\AA^{-1}})$'
    elif dim_x == 'r':
        dim_x = 'r ($\mathrm{\AA})$'
    return dim_x, dim_y

In [23]:
# plot
def _setup_fig():
    if plt.fignum_exists(1):
        plt.clf()
    fig, ax = plt.subplots(num='plot')
    return fig, ax

def _setup_files(files):
    if isinstance(files, str):
        files = [files]
    return files

def _collect_data(files, xmin, xmax):
    # collect data 
    ys = list()
    for fname in files: 
        dat = np.loadtxt(fname).T
        x = dat[0]
        y = dat[1:4]
        x, y = get_x_fitting_range(x, y, [xmin, xmax])
        ys.append(y)
    return x, ys

def _plot(fig, ax, x,y, yshift, data_type, fname):
    # assign
    y, diff, fit = y[0:3]
    
    # plot
    ax.set_title(f"Fit:  {fname}")
    dim_x, dim_y = xy_dim_labels(data_type)
    ax.set_xlabel(dim_x)
    ax.set_ylabel(dim_y)
    ax.set_xlim(x.min(),x.max())

    ## data
    ax.plot(x, np.zeros(len(x)), alpha=0.5, c='C4')
    ax.plot(x, y, alpha=0.6, marker='o', markeredgewidth=1, fillstyle='none', markersize=7,  c="C0", label='data')

    ## fit
    ax.plot(x, fit, alpha=0.8, c="C1", label='fit')

    ## diff
    ax.plot(x, np.zeros(len(x)) + (yshift), alpha=0.7, c='C4')
    ax.plot(x, diff + (yshift), alpha=0.8, c="C2", label='diff')
    
    ax.legend()
    fig.canvas.flush_events()

def _plot_compare(fig, ax, x, ys, yshift, data_type, files, draw_type):
    # assign
    s = 0
    ax.plot(x, np.zeros(len(x)), alpha=0.5, c='C4')
    dim_x, dim_y = xy_dim_labels(data_type)
    ax.set_xlabel(dim_x)
    ax.set_ylabel(dim_y)
    ax.set_xlim(x.min(),x.max())
    
    ax.set_title(f'{draw_type}')
    for y, fname in zip(ys, files):
        if draw_type == 'data': 
            y = y[0]
        if draw_type == 'fit': 
            y = y[2]
        if draw_type == 'diff': 
            y = y[1]
        ax.plot(x, y  + s, alpha=0.8, label=fname)
        s += yshift 
    ax.legend()
    fig.canvas.flush_events()

In [24]:
def draw(fname, data_type:str, xmin:float, xmax:float, yshift:float):
    files = _setup_files(fname)
    fig, ax = _setup_fig()
    x, ys = _collect_data(files, xmin, xmax)
    y = ys[0]
    _plot(fig, ax, x,y, yshift, data_type, fname)
    
def draw_compare_data(compare_all: bool, file1:str, file2:str, draw_type: str,
                      data_type:str, xmin:float, xmax:float, yshift:float):
    files = FILES
    if not compare_all:
        files = [file1] + [file2]
    fig, ax = _setup_fig()
    x, ys = _collect_data(files, xmin, xmax)
    _plot_compare(fig, ax, x, ys, yshift, data_type, files, draw_type)   

In [8]:
def set_dirpath(dirpath):
    os.chdir(os.path.abspath(dirpath))

In [9]:
path = widgets.FileUpload(
    accept='',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=True)
w_path = widgets.HBox([widgets.Label("files :"), path])

In [11]:
interact_manual(set_dirpath, dirpath='')
display(w_path)

interactive(children=(Text(value='', description='dirpath'), Button(description='Run Interact', style=ButtonSt…

In [29]:
# global 
FILES = list(path.value.keys())
print(os.getcwd())
print(FILES)

/home/yr2369/dev/pdfgui_plotter/myplotter
['CoCrNi-A1.dat', 'CoCrNi-A2.dat', 'example.dat']


In [30]:
# get x range
dat = np.loadtxt(FILES[0]).T
x = dat[0]

In [31]:
interactive_draw = interactive(draw, 
                               fname = FILES, 
                               data_type = ['gr', 'fq', 'sq', 'iq'], 
                               xmin=(x.min()-x.min()/10, x.max()), 
                               xmax = (x.min(),x.max()+x.max()/10), 
                               yshift=(-5,5,0.1))

In [32]:
%matplotlib qt

In [33]:
# %matplotlib inline

In [34]:
interactive_draw

interactive(children=(Dropdown(description='fname', options=('CoCrNi-A1.dat', 'CoCrNi-A2.dat', 'example.dat'),…

In [35]:
interactive_draw_compare_data = interactive(draw_compare_data, 
                                            compare_all = False,
                                            file1 = FILES,
                                            file2 = FILES,
                                            draw_type = ['data', 'fit', 'diff'],
                                            data_type = ['gr', 'fq', 'sq', 'iq'], 
                                            xmin=(x.min()-x.min()/10, x.max()), 
                                            xmax = (x.min(),x.max()+x.max()/10), 
                                            yshift=(-5,5,0.1))

In [36]:
interactive_draw_compare_data

interactive(children=(Checkbox(value=False, description='compare_all'), Dropdown(description='file1', options=…